### 0. First load libraries, define train/test split

In [14]:
import numpy as np
import pickle
import csv
import seaborn
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import pairwise_distances
from sklearn.utils import shuffle
from sklearn.linear_model import Ridge
from sklearn.neighbors import KNeighborsRegressor
import warnings
warnings.filterwarnings('ignore')

work_dir = '/Users/ronghao/Mirror/Cornell-Tech/2018-Fa-Course/CS-5785/Homework/Final/'
data_dir = work_dir + 'all/'

In [11]:
# train score does not use discount, dev_dist_mat is pre-discounted, dev_score use discount
Models = {} # Models['ridge']['X']['Y'][rand_state] = (train_score, dev_score, model)
Models['ridge'] = {}

Test_Result = {} # Test_Result['ridge']['X']['Y'] = (test_score, test_dist_mat_origin)
Test_Result['ridge'] = {}

In [3]:
dist_discount_dev_dict = pickle.load(open(work_dir + "modeldata/dist_discount_dev_dict.p", "rb"))
dist_discount_test = pickle.load(open(work_dir + "modeldata/dist_discount_test.p", "rb"))

In [4]:
X_dict = pickle.load(open(work_dir + "modeldata/X_dict.p", "rb"))
Y_dict = pickle.load(open(work_dir + "modeldata/Y_dict.p", "rb"))

In [5]:
X_data = ['only_fc', 'only_pool']#, 'fc_pool'
Y_data = ['full_w2v_mean', 'n_full_w2v_mean', 'a_full_w2v_mean', 'v_full_w2v_mean',\
                          'n_top800_w2v_mean', 'a_top400_w2v_mean', 'v_top400_w2v_mean',\
                          'n_top400_w2v_mean', 'a_top200_w2v_mean', 'v_top200_w2v_mean',\
         'full_bow', 'n_full_bow', 'a_full_bow', 'v_full_bow',\
                     'n_top800_bow', 'a_top400_bow', 'v_top400_bow',\
                     'n_top400_bow', 'a_top200_bow', 'v_top200_bow']

# X_datakey = 'only_pool'
# x_train_dev = X_dict['train_dev'][X_datakey]
# x_test = X_dict['test'][X_datakey]

# Y_datakey = 'full_w2v_mean'
# y_train_dev = Y_dict['train_dev'][Y_datakey]
# y_test = Y_dict['test'][Y_datakey]

# Data Sanity Check

In [6]:
for X_datakey in X_data:
    print(X_dict['train_dev'][X_datakey].shape)
    print(X_dict['test'][X_datakey].shape)

(10000, 1000)
(2000, 1000)
(10000, 2048)
(2000, 2048)


In [7]:
for Y_datakey in Y_data:
    # print(Y_datakey)
    print(Y_dict['train_dev'][Y_datakey].shape)
    print(Y_dict['test'][Y_datakey].shape)

(10000, 300)
(2000, 300)
(10000, 300)
(2000, 300)
(10000, 300)
(2000, 300)
(10000, 300)
(2000, 300)
(10000, 300)
(2000, 300)
(10000, 300)
(2000, 300)
(10000, 300)
(2000, 300)
(10000, 300)
(2000, 300)
(10000, 300)
(2000, 300)
(10000, 300)
(2000, 300)
(10000, 3995)
(2000, 3995)
(10000, 5312)
(2000, 5312)
(10000, 2381)
(2000, 2381)
(10000, 2011)
(2000, 2011)
(10000, 801)
(2000, 801)
(10000, 400)
(2000, 400)
(10000, 400)
(2000, 400)
(10000, 401)
(2000, 401)
(10000, 200)
(2000, 200)
(10000, 200)
(2000, 200)


# Mega For Loop - Ridge Regression

In [29]:
# X_data = ['only_fc']
# Y_data = ['full_bow']

In [8]:
num_train = 8000; num_dev = 2000; num_test = 2000

In [12]:
for X_datakey in X_data:
    Models['ridge'][X_datakey] = {}
    x_train_dev = X_dict['train_dev'][X_datakey]
    x_test = X_dict['test'][X_datakey]
    
    for Y_datakey in Y_data:
        Models['ridge'][X_datakey][Y_datakey] = {}
        y_train_dev = Y_dict['train_dev'][Y_datakey]
        y_test = Y_dict['test'][Y_datakey]
        if 'bow' in Y_datakey:
            y_train_dev = (y_train_dev>0).astype(int) + (y_train_dev>1).astype(int)*0.5
            y_test = (y_test>0).astype(int) + (y_test>1).astype(int)*0.5
        for rand_state in range(4):
            print(X_datakey, Y_datakey, 'rand:', rand_state)
            split_idx = list(range(num_train + num_dev))
            split_idx = shuffle(split_idx, random_state=rand_state)
            x_train = x_train_dev[split_idx[:num_train]]
            x_dev = x_train_dev[split_idx[num_train:]]
            y_train = y_train_dev[split_idx[:num_train]]
            y_dev = y_train_dev[split_idx[num_train:]]
            
            # train OLS model with regression
            ridge_reg = Ridge(alpha=150)
            ridge_reg.fit(x_train, y_train)
            
            # score
            if rand_state == 0:
                y_train_pred = ridge_reg.predict(x_train)
                train_distances = pairwise_distances(y_train, y_train_pred)
                train_scores = []
                train_pos_list = []

                for i in range(num_train):
                    pred_dist_idx = list(np.argsort(train_distances[i]))
                    train_pos = pred_dist_idx.index(i)
                    train_pos_list.append(train_pos)
                    if train_pos < 20:
                        train_scores.append(1 / (train_pos + 1))
                    else:
                        train_scores.append(0.0)
                train_score = np.mean(train_scores)
            else:
                train_score = 0
            
            y_dev_pred = ridge_reg.predict(x_dev)
            dev_distances_origin = pairwise_distances(y_dev, y_dev_pred)
            dev_distances = dev_distances_origin * dist_discount_dev_dict[rand_state]
            dev_scores = []
            dev_pos_list = []

            for i in range(num_dev):
                pred_dist_idx = list(np.argsort(dev_distances[i]))
                dev_pos = pred_dist_idx.index(i)
                dev_pos_list.append(dev_pos)
                if dev_pos < 20:
                    dev_scores.append(1 / (dev_pos + 1))
                else:
                    dev_scores.append(0.0)
            dev_score = np.mean(dev_scores)

            Models['ridge'][X_datakey][Y_datakey][rand_state] =\
                        (train_score, dev_score, ridge_reg)
            print('train_score:', train_score, 'dev_score', dev_score)

only_fc full_w2v_mean rand: 0
train_score: 0.2931911674947859 dev_score 0.3672348454800273
only_fc full_w2v_mean rand: 1
train_score: 0 dev_score 0.3773244888947481
only_fc full_w2v_mean rand: 2
train_score: 0 dev_score 0.3757950045031508
only_fc full_w2v_mean rand: 3
train_score: 0 dev_score 0.37139631378683235
only_fc n_full_w2v_mean rand: 0
train_score: 0.3133915566395249 dev_score 0.3765459752128676
only_fc n_full_w2v_mean rand: 1
train_score: 0 dev_score 0.37683300207059883
only_fc n_full_w2v_mean rand: 2
train_score: 0 dev_score 0.37000333269456726
only_fc n_full_w2v_mean rand: 3
train_score: 0 dev_score 0.36498657768959625
only_fc a_full_w2v_mean rand: 0
train_score: 0.09961369453720943 dev_score 0.20188934014257157
only_fc a_full_w2v_mean rand: 1
train_score: 0 dev_score 0.2067942527072171
only_fc a_full_w2v_mean rand: 2
train_score: 0 dev_score 0.20801152581938187
only_fc a_full_w2v_mean rand: 3
train_score: 0 dev_score 0.21156634176367148
only_fc v_full_w2v_mean rand: 0
train

train_score: 0 dev_score 0.19645004064348107
only_pool a_top400_w2v_mean rand: 3
train_score: 0 dev_score 0.2017908362814516
only_pool v_top400_w2v_mean rand: 0
train_score: 0.1531347962758131 dev_score 0.1852729890444093
only_pool v_top400_w2v_mean rand: 1
train_score: 0 dev_score 0.1854144763346389
only_pool v_top400_w2v_mean rand: 2
train_score: 0 dev_score 0.19249160248076658
only_pool v_top400_w2v_mean rand: 3
train_score: 0 dev_score 0.19482844920172704
only_pool n_top400_w2v_mean rand: 0
train_score: 0.4111224838344275 dev_score 0.35932781010698966
only_pool n_top400_w2v_mean rand: 1
train_score: 0 dev_score 0.35862927380926607
only_pool n_top400_w2v_mean rand: 2
train_score: 0 dev_score 0.35683866108521684
only_pool n_top400_w2v_mean rand: 3
train_score: 0 dev_score 0.35947114946242265
only_pool a_top200_w2v_mean rand: 0
train_score: 0.10255791861056471 dev_score 0.18758570554832163
only_pool a_top200_w2v_mean rand: 1
train_score: 0 dev_score 0.1847243400562286
only_pool a_top2

In [35]:
# Models

{'ridge': {'only_fc': {'full_bow': {0: (0.48742759629807325,
     0.36216435113089523,
     Ridge(alpha=150, copy_X=True, fit_intercept=True, max_iter=None,
        normalize=False, random_state=None, solver='auto', tol=0.001)),
    1: (0, 0.3787090326748415, 'skip'),
    2: (0, 0.38354257917435164, 'skip')}}}}

In [16]:
pickle.dump(Models, open(work_dir + "modeldata/Ridge_train_dev_score.p", "wb"))

### Feed in all the training data for Ridge

In [28]:
X_ridge_data = ['only_fc', 'only_pool']#, 'fc_pool'
Y_ridge_data = ['full_w2v_mean', 'n_full_w2v_mean', 'a_full_w2v_mean', 'v_full_w2v_mean',\
                     'full_bow', 'n_full_bow', 'a_full_bow', 'v_full_bow']

Ridge_all_data_Models = {}# Ridge_Models['X']['Y'] = model

In [29]:
for X_datakey in X_ridge_data:
    Ridge_all_data_Models[X_datakey] = {}
    x_train = X_dict['train_dev'][X_datakey]
    
    for Y_datakey in Y_ridge_data:
        print(X_datakey, Y_datakey)
        y_train = Y_dict['train_dev'][Y_datakey]
        if 'bow' in Y_datakey:
            y_train = (y_train>0).astype(int) + (y_train>1).astype(int)*0.5

        # train OLS model with regression
        ridge_reg = Ridge(alpha=150)
        ridge_reg.fit(x_train, y_train)
        Ridge_all_data_Models[X_datakey][Y_datakey] = ridge_reg

only_fc full_w2v_mean
only_fc n_full_w2v_mean
only_fc a_full_w2v_mean
only_fc v_full_w2v_mean
only_fc full_bow
only_fc n_full_bow
only_fc a_full_bow
only_fc v_full_bow
only_pool full_w2v_mean
only_pool n_full_w2v_mean
only_pool a_full_w2v_mean
only_pool v_full_w2v_mean
only_pool full_bow
only_pool n_full_bow
only_pool a_full_bow
only_pool v_full_bow


In [30]:
pickle.dump(Ridge_all_data_Models, open(work_dir + "modeldata/Ridge_all_data_model.p", "wb"))

### KNN

In [26]:
X_knn_data = ['only_fc', 'only_pool']#, 'fc_pool'
Y_knn_data = ['full_w2v_mean', 'n_full_w2v_mean', 'a_full_w2v_mean', 'v_full_w2v_mean',\
                     'full_bow', 'n_full_bow', 'a_full_bow', 'v_full_bow']

KNN_Models = {} # KNN_Models['X']['Y'][k][rand_state] = (dev_score, model)

In [18]:
num_train = 8000; num_dev = 2000; num_test = 2000

In [34]:
for X_datakey in X_knn_data:
    KNN_Models[X_datakey] = {}
    x_train_dev = X_dict['train_dev'][X_datakey]
    x_test = X_dict['test'][X_datakey]
    
    for Y_datakey in Y_knn_data:
        KNN_Models[X_datakey][Y_datakey] = {}
        y_train_dev = Y_dict['train_dev'][Y_datakey]
        y_test = Y_dict['test'][Y_datakey]
        if 'bow' in Y_datakey:
            y_train_dev = (y_train_dev>0).astype(int) + (y_train_dev>1).astype(int)*0.5
            y_test = (y_test>0).astype(int) + (y_test>1).astype(int)*0.5
        for k in [8]:
                print(X_datakey, Y_datakey, 'K:', k)
                split_idx = list(range(num_train + num_dev))
                split_idx = shuffle(split_idx, random_state=0)
                x_train = x_train_dev[split_idx[:num_train]]
                x_dev = x_train_dev[split_idx[num_train:]]
                y_train = y_train_dev[split_idx[:num_train]]
                y_dev = y_train_dev[split_idx[num_train:]]

                # train OLS model with regression
                neigh = KNeighborsRegressor(n_neighbors=k)
                neigh.fit(x_train, y_train)

                # score
                y_dev_pred = neigh.predict(x_dev)
                dev_distances_origin = pairwise_distances(y_dev, y_dev_pred)
                dev_distances = dev_distances_origin * dist_discount_dev_dict[rand_state]
                dev_scores = []
                dev_pos_list = []

                for i in range(num_dev):
                    pred_dist_idx = list(np.argsort(dev_distances[i]))
                    dev_pos = pred_dist_idx.index(i)
                    dev_pos_list.append(dev_pos)
                    if dev_pos < 20:
                        dev_scores.append(1 / (dev_pos + 1))
                    else:
                        dev_scores.append(0.0)
                dev_score = np.mean(dev_scores)

                KNN_Models[X_datakey][Y_datakey][k] =\
                            (dev_score, ridge_reg)
                print('dev_score', dev_score)

only_fc full_w2v_mean K: 8
dev_score 0.6191734605414365
only_fc n_full_w2v_mean K: 8
dev_score 0.6021428986519156
only_fc a_full_w2v_mean K: 8
dev_score 0.21120994172236435
only_fc v_full_w2v_mean K: 8
dev_score 0.27330411229207674
only_fc full_bow K: 8
dev_score 0.58931275997824
only_fc n_full_bow K: 8
dev_score 0.6217904302160687
only_fc a_full_bow K: 8
dev_score 0.2590453454977255
only_fc v_full_bow K: 8
dev_score 0.3011049905503853
only_pool full_w2v_mean K: 8
dev_score 0.6357816771936353
only_pool n_full_w2v_mean K: 8
dev_score 0.6202575394355386
only_pool a_full_w2v_mean K: 8
dev_score 0.21392581523438722
only_pool v_full_w2v_mean K: 8
dev_score 0.2747523673866553
only_pool full_bow K: 8
dev_score 0.610703214864341
only_pool n_full_bow K: 8
dev_score 0.6321247252317685
only_pool a_full_bow K: 8
dev_score 0.2792457934029335
only_pool v_full_bow K: 8
dev_score 0.302420412125714


In [35]:
pickle.dump(KNN_Models, open(work_dir + "modeldata/KNN_dev_score_model.p", "wb"))

### Feed in all the training data for KNN

In [31]:
X_knn_data = ['only_fc', 'only_pool']#, 'fc_pool'
Y_knn_data = ['full_w2v_mean', 'n_full_w2v_mean', 'a_full_w2v_mean', 'v_full_w2v_mean',\
                     'full_bow', 'n_full_bow', 'a_full_bow', 'v_full_bow']

KNN_all_data_Models = {} # KNN_Models['X']['Y'][k] = model

In [32]:
for X_datakey in X_knn_data:
    KNN_all_data_Models[X_datakey] = {}
    x_train = X_dict['train_dev'][X_datakey]
    
    for Y_datakey in Y_knn_data:
        print(X_datakey, Y_datakey)
        y_train = Y_dict['train_dev'][Y_datakey]
        if 'bow' in Y_datakey:
            y_train = (y_train>0).astype(int) + (y_train>1).astype(int)*0.5

        # train OLS model with regression
        neigh = KNeighborsRegressor(n_neighbors=8)
        neigh.fit(x_train, y_train)
        KNN_all_data_Models[X_datakey][Y_datakey] = neigh

only_fc full_w2v_mean
only_fc n_full_w2v_mean
only_fc a_full_w2v_mean
only_fc v_full_w2v_mean
only_fc full_bow
only_fc n_full_bow
only_fc a_full_bow
only_fc v_full_bow
only_pool full_w2v_mean
only_pool n_full_w2v_mean
only_pool a_full_w2v_mean
only_pool v_full_w2v_mean
only_pool full_bow
only_pool n_full_bow
only_pool a_full_bow
only_pool v_full_bow


In [33]:
pickle.dump(KNN_all_data_Models, open(work_dir + "modeldata/KNN_all_data_Models.p", "wb"))

### 1. Train different models to predict the description from the image features

### 2. Test out model on development data, computing its MAP@20, and investigating the quality of the rankings

In [97]:
# +0.5, alpha=CV, fc
train_dist = test_model_on_train(ridge_reg_best)
print()
dev_dist = test_model_on_dev(ridge_reg_best)

Train MAP@20: 0.4341711737512144
Mean position of true image 21.3235
Median position of true image 3.0

Development MAP@20: 0.3177371248741798
Mean position of true image 17.744
Median position of true image 5.0


In [100]:
# +0.5, alpha=CV, dist_discount, fc
train_dist = test_model_on_train(ridge_reg_best)
print()
dev_dist = test_model_on_dev(ridge_reg_best)

Train MAP@20: 0.4341711737512144
Mean position of true image 21.3235
Median position of true image 3.0

Development MAP@20: 0.37578227602505854
Mean position of true image 13.513
Median position of true image 4.0


In [141]:
# +0.5, alpha=210, dist_discount, pool
train_dist = test_model_on_train(ridge_reg)
print()
dev_dist = test_model_on_dev(ridge_reg)

Train MAP@20: 0.6299895637574284
Mean position of true image 9.29025
Median position of true image 0.0

Development MAP@20: 0.3956479965102837
Mean position of true image 12.853
Median position of true image 4.0


In [143]:
# +0.5, alpha=210, dist_discount, pool
train_dist = test_model_on_train(ridge_reg)
print()
dev_dist = test_model_on_dev(ridge_reg)

Train MAP@20: 0.6299895637574284
Mean position of true image 9.29025
Median position of true image 0.0

Development MAP@20: 0.3956479965102837
Mean position of true image 12.853
Median position of true image 4.0


In [10]:
# +0.5, alpha=210
train_dist = test_model_on_train(ridge_reg)
print()
dev_dist = test_model_on_dev(ridge_reg)

Train MAP@20: 0.6299895637574284
Mean position of true image 9.29025
Median position of true image 0.0

Development MAP@20: 0.3403670830803184
Mean position of true image 16.292
Median position of true image 5.0


In [85]:
# +0.5, alpha=CV
train_dist = test_model_on_train(ridge_reg_best)
print()
dev_dist = test_model_on_dev(ridge_reg_best)

Train MAP@20: 0.6299895637574284
Mean position of true image 9.29025
Median position of true image 0.0

Development MAP@20: 0.3403670830803184
Mean position of true image 16.292
Median position of true image 5.0


In [79]:
# +0.2, alpha=90
train_dist = test_model_on_train(ridge_reg)
print()
dev_dist = test_model_on_dev(ridge_reg)

Train MAP@20: 0.7850865268004054
Mean position of true image 3.858375
Median position of true image 0.0

Development MAP@20: 0.33340503282192524
Mean position of true image 18.2465
Median position of true image 5.0


In [74]:
# +0.5, alpha=90
train_dist = test_model_on_train(ridge_reg)
print()
dev_dist = test_model_on_dev(ridge_reg)

Train MAP@20: 0.7422200892411468
Mean position of true image 4.89675
Median position of true image 0.0

Development MAP@20: 0.34154812759479947
Mean position of true image 17.5565
Median position of true image 5.0


In [69]:
# +0.6, alpha=90
train_dist = test_model_on_train(ridge_reg)
print()
dev_dist = test_model_on_dev(ridge_reg)

Train MAP@20: 0.728070688851697
Mean position of true image 5.247875
Median position of true image 0.0

Development MAP@20: 0.34055813558216813
Mean position of true image 17.567
Median position of true image 4.5


In [35]:
train_dist = test_model_on_train(ridge_reg)
print()
dev_dist = test_model_on_dev(ridge_reg)

Train MAP@20: 0.6828545557705968
Mean position of true image 6.5955
Median position of true image 0.0

Development MAP@20: 0.33738035383733916
Mean position of true image 18.1845
Median position of true image 5.0


In [20]:
train_dist = test_model_on_train(ridge_reg)
print()
dev_dist = test_model_on_dev(ridge_reg)

Train MAP@20: 0.8159132326667784
Mean position of true image 3.18225
Median position of true image 0.0

Development MAP@20: 0.32316708317052745
Mean position of true image 19.8755
Median position of true image 5.0


In [ ]:
# X_data = 'only_fc', 'only_pool', 'fc_pool'
# Y_data = 'full_w2v_mean', 'nv_w2v_mean', 'n_w2v_mean', 'av_w2v_mean', 
#          'BOW_tagEnhanced_1', 'BOW_tagEnhanced_2', 'BOW_tagEnhanced_5'
#          'bow_1294', 'bow_4291'

In [90]:
def overfit_learner(X_data, Y_data):
    x_train_dev = X_dict[X_data]['train_dev']
    x_test = X_dict[X_data]['test']
    y_train_dev = Y_dict[Y_data]['train_dev']
    y_test = Y_dict[Y_data]['test']
    x_train = x_train_dev[split_idx[:num_train]]
    x_dev = x_train_dev[split_idx[num_train:]]
    y_train = y_train_dev[split_idx[:num_train]]
    y_dev = y_train_dev[split_idx[ncombum_train:]]
    
    ridge_reg = Ridge(alpha=90)
    ridge_reg.fit(x_train, y_train)
    
    y_dev_pred = ridge_reg.predict(x_dev)
    dev_distances = pairwise_distances(y_dev, y_dev_pred)
    dev_scores = []
    dev_pos_list = []

    for i in range(num_dev):
        pred_dist_idx = list(np.argsort(dev_distances[i]))
        dev_pos = pred_dist_idx.index(i)
        dev_pos_list.append(dev_pos)
        if dev_pos < 20:
            dev_scores.append(1 / (dev_pos + 1))
        else:
            dev_scores.append(0.0)
    
    return dev_distances, np.mean(dev_scores)

In [122]:
learner_combination = [('only_pool', 'bow_4291'),('only_pool', 'n_w2v_mean'),('only_pool', 'av_w2v_mean'),\
                      ('only_fc', 'bow_4291'),('only_fc', 'n_w2v_mean'),('only_fc', 'av_w2v_mean')]

learners = {}
for comb in learner_combination:
    dist_matrix, score = overfit_learner(comb[0],comb[1])
    learners[comb] = (dist_matrix, score)

In [125]:
learner_weight = {}
for k,v in learners.items():
    learner_weight[k] = v[1]

In [107]:
dev_distances = np.zeros((2000, 2000))
for result in learners.values():
    dev_distances += result[0] * result[1]


dev_scores = []
dev_pos_list = []

for i in range(num_dev):
    pred_dist_idx = list(np.argsort(dev_distances[i]))
    dev_pos = pred_dist_idx.index(i)
    dev_pos_list.append(dev_pos)
    if dev_pos < 20:
        dev_scores.append(1 / (dev_pos + 1))
    else:
        dev_scores.append(0.0)

print("Development MAP@20:", np.mean(dev_scores))
print("Mean position of true image", np.mean(dev_pos_list))
print("Median position of true image", np.median(dev_pos_list))

Development MAP@20: 0.3600585244691669
Mean position of true image 15.89
Median position of true image 4.0


In [ ]:
def inspect_model_performance(image_index, model):
    

### 4. Finally use a model to compute top-20 predictions on the test data that can be submitted to Kaggle

In [126]:
def fullset_learner(X_data, Y_data):
    x_train_dev = X_dict[X_data]['train_dev']
    x_test = X_dict[X_data]['test']
    y_train_dev = Y_dict[Y_data]['train_dev']
    y_test = Y_dict[Y_data]['test']
    
    ridge_reg = Ridge(alpha=90)
    ridge_reg.fit(x_train_dev, y_train_dev)
    
    y_test_pred = ridge_reg.predict(x_test)
    dev_distances = pairwise_distances(y_test, y_test_pred)
    
    return dev_distances

In [127]:
learner_combination = [('only_pool', 'bow_4291'),('only_pool', 'n_w2v_mean'),('only_pool', 'av_w2v_mean'),\
                      ('only_fc', 'bow_4291'),('only_fc', 'n_w2v_mean'),('only_fc', 'av_w2v_mean')]

test_dist = {}
for comb in learner_combination:
    if comb[1]!='av_w2v_mean':
        dist_matrix = fullset_learner(comb[0],comb[1])
        test_dist[comb] = dist_matrix

In [129]:
test_distances = np.zeros((2000, 2000))
for comb in learner_combination:
    if comb[1]!='av_w2v_mean':
        test_distances += test_dist[comb] * learner_weight[comb]

In [130]:
test_distances

array([[5.53844671, 5.63639348, 5.40675457, ..., 5.48945771, 5.91745964,
        6.32867515],
       [5.61504727, 5.61525129, 5.71905203, ..., 5.63511242, 5.73045421,
        4.98068698],
       [3.92571399, 4.13838268, 3.68707737, ..., 3.79472255, 3.94624768,
        4.81389516],
       ...,
       [3.80531132, 4.27495197, 3.91405698, ..., 3.89063719, 4.38231861,
        4.04416667],
       [4.27510325, 4.78701014, 4.38586372, ..., 4.49792071, 4.90364611,
        5.20709396],
       [4.09080414, 3.67006583, 3.65421024, ..., 3.93422112, 4.33886528,
        4.55403355]])

In [131]:
pred_rows = []

for i in range(num_test):
    test_dist_idx = list(np.argsort(test_distances[i]))
    top_20 = test_dist_idx[:20]
    row = ["%d.jpg" % i for i in test_dist_idx[:20]]
    pred_rows.append(" ".join(row))

with open("ojbk_submission.csv", "w") as f:
    f.write("Descritpion_ID,Top_20_Image_IDs\n")
    for i, row in enumerate(pred_rows):
        f.write("%d.txt,%s\n" % (i, row))

print("Output written!")

Output written!


In [16]:
y_test.shape

(2000, 800)

In [25]:
# create test predictions
def output_submission(model):
    x_train_all = np.concatenate([x_train, x_dev])
    y_train_all = np.concatenate([y_train, y_dev])
    model.fit(x_train_all, y_train_all)
    y_test_pred = model.predict(x_test)
    test_distances = pairwise_distances(y_test, y_test_pred)
    pred_rows = []

    for i in range(num_test):
        test_dist_idx = list(np.argsort(test_distances[i]))
        top_20 = test_dist_idx[:20]
        row = ["%d.jpg" % i for i in test_dist_idx[:20]]
        pred_rows.append(" ".join(row))

    with open("test_submission.csv", "w") as f:
        f.write("Descritpion_ID,Top_20_Image_IDs\n")
        for i, row in enumerate(pred_rows):
            f.write("%d.txt,%s\n" % (i, row))

    print("Output written!")

In [27]:
output_submission(ridge_reg)

Output written!


### Discount

In [5]:
desc_file_test = []
for desc_file_index in range(2000):
    # find description
    with open(data_dir+'descriptions_test/'+str(desc_file_index)+'.txt') as f:
        desc_file_test.append(f.read().lower().replace(',',' ').replace('.',' ').replace('\n','').split(' '))

tag_file_test = []
for tag_file_index in range(2000):
    with open(data_dir+'tags_test/'+str(tag_file_index)+'.txt') as f:
        tag_txt = f.read()
    tags = []
    for word in list(pair[pair.find(':')+1:] for pair in tag_txt.split('\n')[:-1]):
        tags += word.split(' ')
    tag_file_test.append(tags)

KeyboardInterrupt: 

In [81]:
def tag_discount_test():
    table = []
    for desc in desc_file_test:
        row = []
        for tags in tag_file_test:
            d_expo = 0
            for tag in tags:
                if tag in desc:
                    d_expo += 1
            row.append(d_expo)
        table.append(row)
    return table

In [82]:
discount_expo_table_test = tag_discount_test()

In [84]:
def discount_table_dev(expo_table, exp_rate=0.8):
    table = []
    for expo_row in expo_table:
        row = []
        for expo in expo_row:
            row.append(exp_rate**expo)
        table.append(row)
    return table

In [85]:
dist_discount_test = np.array(discount_table_dev(discount_expo_table_test, 0.8))

In [86]:
# create test predictions
def output_submission(model):
    x_train_all = np.concatenate([x_train, x_dev])
    y_train_all = np.concatenate([y_train, y_dev])
    model.fit(x_train_all, y_train_all)
    y_test_pred = model.predict(x_test)
    test_distances = pairwise_distances(y_test, y_test_pred) * dist_discount_test
    pred_rows = []

    for i in range(num_test):
        test_dist_idx = list(np.argsort(test_distances[i]))
        top_20 = test_dist_idx[:20]
        row = ["%d.jpg" % i for i in test_dist_idx[:20]]
        pred_rows.append(" ".join(row))

    with open("discount_submission.csv", "w") as f:
        f.write("Descritpion_ID,Top_20_Image_IDs\n")
        for i, row in enumerate(pred_rows):
            f.write("%d.txt,%s\n" % (i, row))

    print("Output written!")

In [ ]:
output_submission(ridge_reg)

In [89]:
x_train_all = np.concatenate([x_train, x_dev])
y_train_all = np.concatenate([y_train, y_dev])
ridge_reg.fit(x_train_all, y_train_all)
y_test_pred = ridge_reg.predict(x_test)
pool_test_distances = pairwise_distances(y_test, y_test_pred) * dist_discount_test

In [102]:
x_train_all = np.concatenate([x_train, x_dev])
y_train_all = np.concatenate([y_train, y_dev])
ridge_reg_best.fit(x_train_all, y_train_all)
y_test_pred = ridge_reg_best.predict(x_test)
fc_test_distances = pairwise_distances(y_test, y_test_pred) * dist_discount_test

In [111]:
ensemble_test_distances = pool_test_distances*0.6702 + fc_test_distances*0.3757

In [112]:
ensemble_test_distances = ensemble_test_distances * dist_discount_test

In [113]:
pred_rows = []

for i in range(num_test):
    test_dist_idx = list(np.argsort(ensemble_test_distances[i]))
    top_20 = test_dist_idx[:20]
    row = ["%d.jpg" % i for i in test_dist_idx[:20]]
    pred_rows.append(" ".join(row))

with open("ensemble_discount_submission.csv", "w") as f:
    f.write("Descritpion_ID,Top_20_Image_IDs\n")
    for i, row in enumerate(pred_rows):
        f.write("%d.txt,%s\n" % (i, row))

print("Output written!")

Output written!


In [136]:
0.8 / 0.8**0.6

0.9146101038546527

In [137]:
pool_test_distances_maga = pool_test_distances / dist_discount_test**0.6
pred_rows = []

for i in range(num_test):
    test_dist_idx = list(np.argsort(pool_test_distances_maga[i]))
    top_20 = test_dist_idx[:20]
    row = ["%d.jpg" % i for i in test_dist_idx[:20]]
    pred_rows.append(" ".join(row))

with open("pool_discount_maga_submission.csv", "w") as f:
    f.write("Descritpion_ID,Top_20_Image_IDs\n")
    for i, row in enumerate(pred_rows):
        f.write("%d.txt,%s\n" % (i, row))

print("Output written!")

Output written!
